In [3]:
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.storage import StorageManagementClient
import os
from dotenv import load_dotenv
import requests
import json

# Load environment variables
load_dotenv()

print("="*60)
print("SERVICE PRINCIPAL PERMISSIONS CHECK")
print("="*60)

# Get SP details
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")
tenant_id = os.getenv("AZURE_TENANT_ID")
subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace_name = os.getenv("AZUREML_WORKSPACE_NAME")

print(f"Service Principal Client ID: {client_id}")
print(f"Tenant ID: {tenant_id}")
print(f"Subscription: {subscription_id}")
print(f"Resource Group: {resource_group}")
print()

# Create credential
credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

# ==============================================================================
# Get Service Principal Object ID using Graph API
# ==============================================================================
print("🔍 Getting Service Principal Object ID...")
print("-"*40)

def get_sp_object_id():
    """Get the Object ID of the Service Principal using Graph API"""
    
    try:
        # Get access token for Microsoft Graph
        token_response = credential.get_token("https://graph.microsoft.com/.default")
        access_token = token_response.token
        
        # Call Graph API to get service principal
        headers = {
            'Authorization': f'Bearer {access_token}',
            'Content-Type': 'application/json'
        }
        
        # Search for service principal by appId (client_id)
        url = f"https://graph.microsoft.com/v1.0/servicePrincipals?$filter=appId eq '{client_id}'"
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        if data.get('value') and len(data['value']) > 0:
            sp_object_id = data['value'][0]['id']
            sp_display_name = data['value'][0].get('displayName', 'N/A')
            print(f"✓ Found Service Principal:")
            print(f"  Display Name: {sp_display_name}")
            print(f"  Object ID: {sp_object_id}")
            print(f"  Application ID: {client_id}")
            return sp_object_id
        else:
            print(f"⚠️ Service Principal not found for Client ID: {client_id}")
            return None
            
    except Exception as e:
        print(f"⚠️ Could not get SP Object ID via Graph API: {str(e)}")
        print("  This is OK, we'll check using other methods.")
        return None

sp_object_id = get_sp_object_id()

SERVICE PRINCIPAL PERMISSIONS CHECK
Service Principal Client ID: e77fcc8e-5551-47b6-a600-1d5633c81e31
Tenant ID: 3540e7dc-31b3-4057-9e31-43e9fe938179
Subscription: b8d6d487-0bd2-4773-b318-12ab763ed178
Resource Group: strategicai-rg-uks-dev-01

🔍 Getting Service Principal Object ID...
----------------------------------------
✓ Found Service Principal:
  Display Name: strategicai-svp-app-uks-dev-01
  Object ID: 3ac05fe8-0fcc-416a-b36a-832d96719b5b
  Application ID: e77fcc8e-5551-47b6-a600-1d5633c81e31


In [4]:
# ==============================================================================
# Check Role Assignments using the correct SDK methods
# ==============================================================================
print("\n🔍 Checking Service Principal's Role Assignments...")
print("-"*40)

auth_client = AuthorizationManagementClient(credential, subscription_id)

# Storage account we're interested in
storage_account_scope = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Storage/storageAccounts/strategicaistuksdev02"
ml_workspace_scope = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}"
resource_group_scope = f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}"

found_roles = {
    'storage_blob_contributor': False,
    'storage_account_contributor': False,
    'ml_contributor': False,
    'rg_contributor': False,
    'all_roles': []
}

try:
    # Method 1: List role assignments for the subscription
    print("\nChecking role assignments at subscription level...")
    
    # List all role assignments in the subscription
    assignments = auth_client.role_assignments.list_for_subscription()
    
    count = 0
    for assignment in assignments:
        count += 1
        # Check if this assignment is for our Service Principal (by client ID)
        # We need to check if the principal_id matches our SP
        if sp_object_id and assignment.principal_id == sp_object_id:
            try:
                # Get role definition details
                role_def = auth_client.role_definitions.get_by_id(assignment.role_definition_id)
                
                role_info = {
                    'role_name': role_def.role_name,
                    'scope': assignment.scope,
                    'principal_id': assignment.principal_id
                }
                
                found_roles['all_roles'].append(role_info)
                
                # Check for specific roles
                if 'Storage Blob Data Contributor' in role_def.role_name:
                    if 'strategicaistuksdev02' in assignment.scope or resource_group in assignment.scope:
                        found_roles['storage_blob_contributor'] = True
                        
                if 'Contributor' in role_def.role_name:
                    if resource_group in assignment.scope:
                        found_roles['rg_contributor'] = True
                        
            except Exception as e:
                continue
    
    print(f"  Checked {count} total role assignments in subscription")
    
except Exception as e:
    print(f"  Method 1 failed: {str(e)}")


🔍 Checking Service Principal's Role Assignments...
----------------------------------------

Checking role assignments at subscription level...
  Checked 89 total role assignments in subscription


In [5]:
# Method 2: Check specific scopes
try:
    print("\nChecking role assignments at resource group level...")
    
    # List role assignments for the resource group
    rg_assignments = auth_client.role_assignments.list_for_resource_group(resource_group)
    
    for assignment in rg_assignments:
        if sp_object_id and assignment.principal_id == sp_object_id:
            try:
                role_def = auth_client.role_definitions.get_by_id(assignment.role_definition_id)
                
                role_info = {
                    'role_name': role_def.role_name,
                    'scope': 'Resource Group',
                    'principal_id': assignment.principal_id
                }
                
                # Avoid duplicates
                if not any(r['role_name'] == role_info['role_name'] and r['scope'] == 'Resource Group' for r in found_roles['all_roles']):
                    found_roles['all_roles'].append(role_info)
                
                if 'Storage Blob Data Contributor' in role_def.role_name:
                    found_roles['storage_blob_contributor'] = True
                if 'Contributor' in role_def.role_name:
                    found_roles['rg_contributor'] = True
                    
            except Exception as e:
                continue
                
    print(f"  Found {len(found_roles['all_roles'])} role assignments for the Service Principal")
    
except Exception as e:
    print(f"  Method 2 failed: {str(e)}")


Checking role assignments at resource group level...
  Found 20 role assignments for the Service Principal


In [6]:
# ==============================================================================
# ANALYSIS AND RECOMMENDATIONS
# ==============================================================================
print("\n" + "="*60)
print("📊 PERMISSION ANALYSIS")
print("="*60)

print("\nCritical Permissions Status:")
print("-"*40)

# Check for Storage Blob Data Contributor
if found_roles['storage_blob_contributor']:
    print("✅ Storage Blob Data Contributor: FOUND")
    print("   Your SP has the required storage permissions")
else:
    print("❌ Storage Blob Data Contributor: NOT FOUND")
    print("   This is causing the AuthorizationFailure error!")

# Check for Resource Group Contributor
if found_roles['rg_contributor']:
    print("✅ Resource Group Contributor: FOUND")
else:
    print("⚠️ Resource Group Contributor: NOT FOUND")
    print("   You may have limited permissions")



📊 PERMISSION ANALYSIS

Critical Permissions Status:
----------------------------------------
✅ Storage Blob Data Contributor: FOUND
   Your SP has the required storage permissions
✅ Resource Group Contributor: FOUND
